# Advanced Analyses: Leveraging Colab Pro Compute

This notebook implements computationally intensive analyses that benefit from Colab Pro's TPU/GPU resources.

## Contents
1. **Stacked DiD** - Addresses TWFE bias (Callaway-Sant'Anna, Sun-Abraham)
2. **Synthetic Control Method** - Alternative identification
3. **Placebo Tests in Time** - Validate no pre-trend effects
4. **Bootstrap Inference** - Robust confidence intervals
5. **Matched DiD** - Propensity score matching + DiD
6. **Additional Outcome Variables** - What else is "hollowing"?

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install packages
!pip install linearmodels pyfixest scikit-learn joblib tqdm -q
print("Packages installed!")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import statsmodels.api as sm
from linearmodels.panel import PanelOLS
from scipy import stats
from joblib import Parallel, delayed
import multiprocessing
from tqdm.notebook import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

N_CORES = multiprocessing.cpu_count()
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

DATA_PATH = Path('/content/drive/MyDrive/Paper_2')

print(f"CPU cores: {N_CORES}")

In [ ]:
# Load analysis panel from previous notebook
try:
    panel = pd.read_parquet(DATA_PATH / 'analysis_panel_hollow_firm.parquet')
    print(f"Loaded panel: {panel.shape}")
except:
    print("Run notebook 04 first to create analysis panel")
    raise

# Identify firm ID column
FIRM_ID = 'firm_id' if 'firm_id' in panel.columns else panel.columns[0]
print(f"Firm ID: {FIRM_ID}")
print(f"Firms: {panel[FIRM_ID].nunique():,}")

---
## 1. Placebo Tests in Time

**Purpose:** Test whether we find "effects" at fake treatment dates (before ChatGPT).

If the true effect is causal, we should find no significant effects at placebo dates.

In [ ]:
def run_placebo_time_test(df, outcome, firm_id, time_var='period',
                          true_event_yq=2022*4+4):
    """
    Run DiD at multiple placebo event dates.
    Only use pre-treatment data to avoid contamination.
    """
    # Use only pre-treatment data
    pre_data = df[df['yearquarter'] < true_event_yq].copy()
    
    # Get available quarters for placebo tests
    quarters = sorted(pre_data['yearquarter'].unique())
    
    # Test placebo dates (leaving 4 quarters pre and post for each test)
    placebo_quarters = quarters[4:-4]
    
    results = []
    
    for placebo_yq in tqdm(placebo_quarters, desc="Placebo time tests"):
        # Create placebo post indicator
        test_data = pre_data.copy()
        test_data['post_placebo'] = (test_data['yearquarter'] >= placebo_yq).astype(int)
        test_data['treat_x_post_placebo'] = test_data['treated'] * test_data['post_placebo']
        
        try:
            reg_data = test_data[[firm_id, time_var, outcome, 'treat_x_post_placebo']].dropna()
            reg_data = reg_data.set_index([firm_id, time_var])
            
            y = reg_data[outcome]
            X = sm.add_constant(reg_data[['treat_x_post_placebo']])
            
            model = PanelOLS(y, X, entity_effects=True, time_effects=True)
            result = model.fit(cov_type='clustered', cluster_entity=True)
            
            year = (placebo_yq - 1) // 4
            quarter = ((placebo_yq - 1) % 4) + 1
            
            results.append({
                'placebo_date': f"{year}Q{quarter}",
                'yearquarter': placebo_yq,
                'coef': result.params['treat_x_post_placebo'],
                'se': result.std_errors['treat_x_post_placebo'],
                'pval': result.pvalues['treat_x_post_placebo'],
                'nobs': result.nobs
            })
        except:
            pass
    
    return pd.DataFrame(results)

# Run placebo time tests
print("\n" + "="*60)
print("PLACEBO TESTS IN TIME")
print("="*60)

placebo_time_results = run_placebo_time_test(
    panel, 'sga_efficiency', FIRM_ID, 'period'
)

print(f"\nPlacebo tests completed: {len(placebo_time_results)}")
print(f"Significant at 5%: {(placebo_time_results['pval'] < 0.05).sum()}")
print(f"Expected by chance: {len(placebo_time_results) * 0.05:.1f}")

In [ ]:
# Plot placebo time test results
def plot_placebo_time_tests(placebo_results, true_date='2022Q4'):
    """
    Plot coefficients from placebo time tests.
    """
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # Plot coefficients with CIs
    x = range(len(placebo_results))
    
    ax.errorbar(x, placebo_results['coef'], 
                yerr=1.96 * placebo_results['se'],
                fmt='o', color='#3498DB', capsize=3, capthick=1,
                label='Placebo Coefficients')
    
    # Reference line at 0
    ax.axhline(0, color='black', linestyle='-', linewidth=0.5)
    
    # Mark significant results
    sig_mask = placebo_results['pval'] < 0.05
    if sig_mask.any():
        ax.scatter(np.array(x)[sig_mask], placebo_results.loc[sig_mask, 'coef'],
                   color='#E74C3C', s=100, marker='*', zorder=5,
                   label='Significant at 5%')
    
    ax.set_xticks(x[::4])  # Show every 4th label
    ax.set_xticklabels(placebo_results['placebo_date'].iloc[::4], rotation=45)
    
    ax.set_xlabel('Placebo Event Date', fontsize=12)
    ax.set_ylabel('DiD Coefficient', fontsize=12)
    ax.set_title('Figure 4: Placebo Tests in Time\n(Pre-ChatGPT Period Only)', 
                 fontsize=14, fontweight='bold')
    
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

if len(placebo_time_results) > 0:
    fig = plot_placebo_time_tests(placebo_time_results)
    plt.show()

---
## 2. Bootstrap Confidence Intervals

More robust inference through bootstrap (cluster bootstrap at firm level).

In [ ]:
def bootstrap_did_single(df, outcome, firm_id, time_var, seed):
    """
    Single bootstrap iteration with cluster resampling.
    """
    np.random.seed(seed)
    
    try:
        # Cluster bootstrap: resample firms with replacement
        firms = df[firm_id].unique()
        boot_firms = np.random.choice(firms, size=len(firms), replace=True)
        
        # Create bootstrap sample
        boot_dfs = []
        for i, firm in enumerate(boot_firms):
            firm_data = df[df[firm_id] == firm].copy()
            firm_data['boot_firm_id'] = i  # New ID to handle duplicates
            boot_dfs.append(firm_data)
        
        boot_df = pd.concat(boot_dfs, ignore_index=True)
        
        # Run regression
        reg_data = boot_df[['boot_firm_id', time_var, outcome, 'treated_x_post']].dropna()
        reg_data = reg_data.set_index(['boot_firm_id', time_var])
        
        y = reg_data[outcome]
        X = sm.add_constant(reg_data[['treated_x_post']])
        
        model = PanelOLS(y, X, entity_effects=True, time_effects=True)
        result = model.fit()
        
        return result.params['treated_x_post']
    
    except:
        return np.nan


def bootstrap_confidence_intervals(df, outcome, firm_id, time_var='period',
                                   n_bootstrap=2000, n_jobs=-1):
    """
    Compute bootstrap confidence intervals.
    """
    print(f"\nRunning {n_bootstrap:,} bootstrap iterations...")
    
    seeds = np.random.randint(0, 1e7, n_bootstrap)
    
    boot_coefs = Parallel(n_jobs=n_jobs, verbose=5)(
        delayed(bootstrap_did_single)(df, outcome, firm_id, time_var, seed)
        for seed in seeds
    )
    
    boot_coefs = np.array([c for c in boot_coefs if not np.isnan(c)])
    
    # Compute confidence intervals
    ci_percentile = np.percentile(boot_coefs, [2.5, 97.5])
    
    # Bias-corrected accelerated (BCa) would be better but more complex
    
    return boot_coefs, ci_percentile

# Run bootstrap
print("\n" + "="*60)
print("BOOTSTRAP CONFIDENCE INTERVALS")
print("="*60)

boot_coefs, boot_ci = bootstrap_confidence_intervals(
    panel, 'sga_efficiency', FIRM_ID, 'period',
    n_bootstrap=2000, n_jobs=N_CORES
)

print(f"\nBootstrap Results:")
print(f"  Mean: {boot_coefs.mean():.6f}")
print(f"  Std Dev: {boot_coefs.std():.6f}")
print(f"  95% CI: [{boot_ci[0]:.6f}, {boot_ci[1]:.6f}]")

---
## 3. Propensity Score Matching + DiD

Match treated and control firms on pre-treatment characteristics, then run DiD on matched sample.

In [ ]:
def propensity_score_matching(df, firm_id, treatment_col='treated',
                               covariates=None, n_neighbors=1):
    """
    Propensity score matching at firm level using pre-treatment characteristics.
    """
    # Get pre-period firm-level characteristics
    pre_data = df[df['post'] == 0]
    
    # Find available covariates
    if covariates is None:
        potential = ['log_revenue', 'log_employees', 'log_assets', 'sga_efficiency']
        covariates = [c for c in potential if c in pre_data.columns]
    
    print(f"Matching on: {covariates}")
    
    # Compute firm-level averages
    firm_chars = pre_data.groupby(firm_id).agg({
        treatment_col: 'first',
        **{c: 'mean' for c in covariates}
    }).reset_index()
    
    firm_chars = firm_chars.dropna()
    
    print(f"Firms for matching: {len(firm_chars)}")
    print(f"  Treated: {(firm_chars[treatment_col]==1).sum()}")
    print(f"  Control: {(firm_chars[treatment_col]==0).sum()}")
    
    # Estimate propensity scores
    X = firm_chars[covariates]
    y = firm_chars[treatment_col]
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Logistic regression for propensity scores
    ps_model = LogisticRegression(random_state=RANDOM_SEED, max_iter=1000)
    ps_model.fit(X_scaled, y)
    
    firm_chars['propensity_score'] = ps_model.predict_proba(X_scaled)[:, 1]
    
    # Nearest neighbor matching
    treated = firm_chars[firm_chars[treatment_col] == 1]
    control = firm_chars[firm_chars[treatment_col] == 0]
    
    # Fit NN on control propensity scores
    nn = NearestNeighbors(n_neighbors=n_neighbors, metric='euclidean')
    nn.fit(control[['propensity_score']])
    
    # Find matches for treated
    distances, indices = nn.kneighbors(treated[['propensity_score']])
    
    # Get matched control firm IDs
    matched_control_ids = control.iloc[indices.flatten()][firm_id].values
    treated_ids = treated[firm_id].values
    
    # Combine matched sample
    matched_firms = np.concatenate([treated_ids, matched_control_ids])
    
    print(f"\nMatched sample: {len(matched_firms)} firms")
    
    return matched_firms, firm_chars

# Run matching
print("\n" + "="*60)
print("PROPENSITY SCORE MATCHING")
print("="*60)

matched_firms, firm_chars = propensity_score_matching(panel, FIRM_ID)

In [ ]:
# Run DiD on matched sample
matched_panel = panel[panel[FIRM_ID].isin(matched_firms)].copy()

print(f"\nMatched sample:")
print(f"  Observations: {len(matched_panel):,}")
print(f"  Firms: {matched_panel[FIRM_ID].nunique():,}")

# Run DiD
reg_data = matched_panel[[FIRM_ID, 'period', 'sga_efficiency', 'treated_x_post']].dropna()
reg_data = reg_data.set_index([FIRM_ID, 'period'])

y = reg_data['sga_efficiency']
X = sm.add_constant(reg_data[['treated_x_post']])

model = PanelOLS(y, X, entity_effects=True, time_effects=True)
matched_result = model.fit(cov_type='clustered', cluster_entity=True)

print("\nMatched DiD Results:")
print(matched_result.summary.tables[1])

In [ ]:
# Check covariate balance after matching
def check_balance(df, firm_chars, matched_firms, firm_id, covariates):
    """
    Check covariate balance before and after matching.
    """
    # Before matching
    before = firm_chars.copy()
    
    # After matching
    after = firm_chars[firm_chars[firm_id].isin(matched_firms)].copy()
    
    balance_results = []
    
    for cov in covariates:
        if cov not in firm_chars.columns:
            continue
            
        # Before
        t_before = before[before['treated']==1][cov]
        c_before = before[before['treated']==0][cov]
        std_diff_before = (t_before.mean() - c_before.mean()) / np.sqrt((t_before.var() + c_before.var())/2)
        
        # After
        t_after = after[after['treated']==1][cov]
        c_after = after[after['treated']==0][cov]
        std_diff_after = (t_after.mean() - c_after.mean()) / np.sqrt((t_after.var() + c_after.var())/2)
        
        balance_results.append({
            'Covariate': cov,
            'Std Diff (Before)': std_diff_before,
            'Std Diff (After)': std_diff_after,
            'Improvement': abs(std_diff_before) - abs(std_diff_after)
        })
    
    return pd.DataFrame(balance_results)

covariates = ['log_revenue', 'log_employees', 'sga_efficiency']
covariates = [c for c in covariates if c in firm_chars.columns]

balance = check_balance(panel, firm_chars, matched_firms, FIRM_ID, covariates)
print("\nCovariate Balance:")
display(balance.round(4))

---
## 4. Industry Leave-One-Out Robustness

Check that results aren't driven by a single industry.

In [ ]:
def leave_one_out_industry(df, outcome, firm_id, time_var='period',
                           industry_col=None):
    """
    Re-run DiD excluding each industry one at a time.
    """
    if industry_col is None or industry_col not in df.columns:
        # Try to find industry column
        for col in df.columns:
            if 'industry' in col.lower() or 'sector' in col.lower():
                industry_col = col
                break
    
    if industry_col is None:
        print("No industry column found")
        return None
    
    industries = df[industry_col].dropna().unique()
    print(f"Testing {len(industries)} industries")
    
    results = []
    
    for industry in tqdm(industries, desc="Leave-one-out"):
        subset = df[df[industry_col] != industry]
        
        try:
            reg_data = subset[[firm_id, time_var, outcome, 'treated_x_post']].dropna()
            reg_data = reg_data.set_index([firm_id, time_var])
            
            y = reg_data[outcome]
            X = sm.add_constant(reg_data[['treated_x_post']])
            
            model = PanelOLS(y, X, entity_effects=True, time_effects=True)
            result = model.fit(cov_type='clustered', cluster_entity=True)
            
            results.append({
                'excluded_industry': str(industry)[:50],
                'coef': result.params['treated_x_post'],
                'se': result.std_errors['treated_x_post'],
                'pval': result.pvalues['treated_x_post'],
                'nobs': result.nobs
            })
        except:
            pass
    
    return pd.DataFrame(results)

# Run leave-one-out
print("\n" + "="*60)
print("LEAVE-ONE-OUT INDUSTRY ROBUSTNESS")
print("="*60)

loo_results = leave_one_out_industry(panel, 'sga_efficiency', FIRM_ID, 'period')

if loo_results is not None and len(loo_results) > 0:
    print(f"\nCoefficient range: [{loo_results['coef'].min():.6f}, {loo_results['coef'].max():.6f}]")
    print(f"All coefficients significant: {(loo_results['pval'] < 0.05).all()}")

In [ ]:
# Plot leave-one-out results
if loo_results is not None and len(loo_results) > 0:
    fig, ax = plt.subplots(figsize=(12, 8))
    
    loo_sorted = loo_results.sort_values('coef')
    
    y_pos = range(len(loo_sorted))
    
    ax.barh(y_pos, loo_sorted['coef'], xerr=1.96*loo_sorted['se'],
            color='#3498DB', alpha=0.7, capsize=2)
    
    ax.axvline(0, color='black', linewidth=0.5)
    
    # Show baseline
    baseline = panel[[FIRM_ID, 'period', 'sga_efficiency', 'treated_x_post']].dropna()
    baseline = baseline.set_index([FIRM_ID, 'period'])
    baseline_model = PanelOLS(baseline['sga_efficiency'], 
                               sm.add_constant(baseline[['treated_x_post']]),
                               entity_effects=True, time_effects=True)
    baseline_coef = baseline_model.fit().params['treated_x_post']
    ax.axvline(baseline_coef, color='#E74C3C', linewidth=2, linestyle='--',
               label=f'Full sample: {baseline_coef:.4f}')
    
    ax.set_yticks(y_pos)
    ax.set_yticklabels(loo_sorted['excluded_industry'], fontsize=8)
    
    ax.set_xlabel('DiD Coefficient', fontsize=12)
    ax.set_title('Figure 5: Leave-One-Out Industry Robustness', fontsize=14, fontweight='bold')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.show()

---
## 5. Alternative Outcome Variables

What else might be "hollowing"?

In [ ]:
# Create additional outcome variables
def create_additional_outcomes(df, var_map):
    """
    Create additional outcome variables for robustness.
    """
    df = df.copy()
    
    # Find available columns
    revenue_col = None
    employee_col = None
    ebitda_col = None
    
    for col in df.columns:
        col_lower = col.lower()
        if ('revenue' in col_lower or 'sales' in col_lower) and revenue_col is None:
            revenue_col = col
        elif 'employee' in col_lower and employee_col is None:
            employee_col = col
        elif 'ebitda' in col_lower and ebitda_col is None:
            ebitda_col = col
    
    # Revenue growth (if we have lagged values)
    if revenue_col:
        df['revenue_growth'] = df.groupby(FIRM_ID)[revenue_col].pct_change(4)  # YoY
        df['revenue_growth'] = winsorize(df['revenue_growth'], 0.01, 0.99)
    
    # Employee intensity (employees / revenue)
    if employee_col and revenue_col:
        mask = (df[revenue_col] > 0)
        df['employee_intensity'] = np.nan
        df.loc[mask, 'employee_intensity'] = df.loc[mask, employee_col] / df.loc[mask, revenue_col] * 1e6
        df['employee_intensity'] = winsorize(df['employee_intensity'], 0.01, 0.99)
    
    return df

def winsorize(series, lower=0.01, upper=0.99):
    lower_bound = series.quantile(lower)
    upper_bound = series.quantile(upper)
    return series.clip(lower=lower_bound, upper=upper_bound)

# Create additional outcomes
panel = create_additional_outcomes(panel, None)
print("Additional outcomes created")

In [ ]:
# Run DiD for all outcomes
outcomes = [
    ('sga_efficiency', 'SG&A Efficiency (SG&A/Revenue)'),
    ('revenue_per_employee', 'Revenue per Employee'),
    ('ebitda_margin', 'EBITDA Margin'),
    ('employee_intensity', 'Employee Intensity (Emp/Rev)'),
    ('revenue_growth', 'Revenue Growth (YoY)')
]

outcomes = [(var, label) for var, label in outcomes if var in panel.columns]

print("\n" + "="*80)
print("TABLE 5: MULTIPLE OUTCOME ANALYSIS")
print("="*80)

multi_outcome_results = []

for var, label in outcomes:
    try:
        reg_data = panel[[FIRM_ID, 'period', var, 'treated_x_post']].dropna()
        if len(reg_data) < 100:
            continue
            
        reg_data = reg_data.set_index([FIRM_ID, 'period'])
        
        y = reg_data[var]
        X = sm.add_constant(reg_data[['treated_x_post']])
        
        model = PanelOLS(y, X, entity_effects=True, time_effects=True)
        result = model.fit(cov_type='clustered', cluster_entity=True)
        
        multi_outcome_results.append({
            'Outcome': label,
            'β': result.params['treated_x_post'],
            'SE': result.std_errors['treated_x_post'],
            't-stat': result.tstats['treated_x_post'],
            'p-value': result.pvalues['treated_x_post'],
            'N': result.nobs,
            'R² (within)': result.rsquared_within
        })
    except Exception as e:
        print(f"Error for {label}: {e}")

if multi_outcome_results:
    results_df = pd.DataFrame(multi_outcome_results)
    display(results_df.round(4))

---
## 6. Summary of All Robustness Checks

In [ ]:
# Compile all results
print("\n" + "="*80)
print("ROBUSTNESS SUMMARY")
print("="*80)

print("""
┌─────────────────────────────────────────────────────────────────────────────┐
│                      ROBUSTNESS CHECK SUMMARY                               │
├─────────────────────────────────────────────────────────────────────────────┤""")

# Main result
main_data = panel[[FIRM_ID, 'period', 'sga_efficiency', 'treated_x_post']].dropna()
main_data = main_data.set_index([FIRM_ID, 'period'])
main_model = PanelOLS(main_data['sga_efficiency'], 
                       sm.add_constant(main_data[['treated_x_post']]),
                       entity_effects=True, time_effects=True)
main_result = main_model.fit(cov_type='clustered', cluster_entity=True)

print(f"│  1. Main DiD Result                β = {main_result.params['treated_x_post']:>10.6f}  p = {main_result.pvalues['treated_x_post']:.4f}  │")

# Bootstrap CI
if 'boot_ci' in dir():
    print(f"│  2. Bootstrap 95% CI               [{boot_ci[0]:>10.6f}, {boot_ci[1]:.6f}]          │")

# Matched sample
if 'matched_result' in dir():
    print(f"│  3. Matched Sample DiD             β = {matched_result.params['treated_x_post']:>10.6f}  p = {matched_result.pvalues['treated_x_post']:.4f}  │")

# Placebo tests
if 'placebo_time_results' in dir() and len(placebo_time_results) > 0:
    sig_rate = (placebo_time_results['pval'] < 0.05).mean() * 100
    print(f"│  4. Placebo Time Tests             {sig_rate:>5.1f}% significant (expect 5%)           │")

# Leave-one-out
if 'loo_results' in dir() and loo_results is not None:
    all_sig = (loo_results['pval'] < 0.05).all()
    status = "✓ All significant" if all_sig else "✗ Some not significant"
    print(f"│  5. Leave-One-Out Industry         {status:>30}  │")

print("└─────────────────────────────────────────────────────────────────────────────┘")

In [ ]:
# Save all results
panel.to_parquet(DATA_PATH / 'analysis_panel_full.parquet', index=False)

if 'boot_coefs' in dir():
    np.save(DATA_PATH / 'bootstrap_coefficients.npy', boot_coefs)

if 'placebo_time_results' in dir() and placebo_time_results is not None:
    placebo_time_results.to_csv(DATA_PATH / 'placebo_time_results.csv', index=False)

if 'loo_results' in dir() and loo_results is not None:
    loo_results.to_csv(DATA_PATH / 'leave_one_out_results.csv', index=False)

print("\n✓ All results saved to Google Drive")